In [4]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Bidirectional, Embedding, LSTM, Dropout, Dense
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, roc_auc_score

# Define custom Hindi stopwords
stopwords_hindi = [
    'अत', 'अपना', 'अपनी', 'अपने', 'अभी', 'अंदर', 'आदि', 'आप', 'इत्यादि', 'इन', 'इनका', 'इन्हीं', 'इन्हें', 'इन्हों',
    'इस', 'इसका', 'इसकी', 'इसके', 'इसमें', 'इसी', 'इसे', 'उन', 'उनका', 'उनकी', 'उनके', 'उनको', 'उन्हीं', 'उन्हें',
    'उन्हों', 'उस', 'उसके', 'उसी', 'उसे', 'एक', 'एवं', 'एस', 'ऐसे', 'और', 'कई', 'कर', 'करता', 'करते', 'करना',
    'करने', 'करें', 'कहते', 'कहा', 'का', 'काफ़ी', 'कि', 'कितना', 'किन्हें', 'किन्हों', 'किया', 'किर', 'किस',
    'किसी', 'किसे', 'की', 'कुछ', 'कुल', 'के', 'को', 'कोई', 'कौन', 'कौन', 'बही', 'बहुत', 'बाद', 'बाला', 'बिलकुल',
    'भी', 'भीतर', 'मगर', 'मानो', 'मे', 'में', 'यदि', 'यह', 'यहाँ', 'यही', 'या', 'यिह', 'ये', 'रखें', 'रहा', 'रहे',
    'ऱ्वासा', 'लिए', 'लिये', 'लेकिन', 'व', 'वग़ैरह', 'वर्ग', 'वह', 'वहाँ', 'वहीं', 'वाले', 'वुह', 'वे', 'सकता',
    'सकते', 'सबसे', 'सभी', 'साथ', 'साबुत', 'साभ', 'सारा', 'से', 'सो', 'संग', 'ही', 'हुआ', 'हुई', 'हुए', 'है',
    'हैं', 'हो', 'होता', 'होती', 'होते', 'होना', 'होने'
]

# Load dataset
data = pd.read_csv('/content/randomized_combined_dataset.csv')

# Drop null values
data = data.dropna()

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences

# Define stopwords for both Hindi and English
stopwords_hindi = set(["
    'अत', 'अपना', 'अपनी', 'अपने', 'अभी', 'अंदर', 'आदि', 'आप', 'इत्यादि', 'इन', 'इनका', 'इन्हीं', 'इन्हें', 'इन्हों',
    'इस', 'इसका', 'इसकी', 'इसके', 'इसमें', 'इसी', 'इसे', 'उन', 'उनका', 'उनकी', 'उनके', 'उनको', 'उन्हीं', 'उन्हें',
    'उन्हों', 'उस', 'उसके', 'उसी', 'उसे', 'एक', 'एवं', 'एस', 'ऐसे', 'और', 'कई', 'कर', 'करता', 'करते', 'करना',
    'करने', 'करें', 'कहते', 'कहा', 'का', 'काफ़ी', 'कि', 'कितना', 'किन्हें', 'किन्हों', 'किया', 'किर', 'किस',
    'किसी', 'किसे', 'की', 'कुछ', 'कुल', 'के', 'को', 'कोई', 'कौन', 'कौन', 'बही', 'बहुत', 'बाद', 'बाला', 'बिलकुल',
    'भी', 'भीतर', 'मगर', 'मानो', 'मे', 'में', 'यदि', 'यह', 'यहाँ', 'यही', 'या', 'यिह', 'ये', 'रखें', 'रहा', 'रहे',
    'ऱ्वासा', 'लिए', 'लिये', 'लेकिन', 'व', 'वग़ैरह', 'वर्ग', 'वह', 'वहाँ', 'वहीं', 'वाले', 'वुह', 'वे', 'सकता',
    'सकते', 'सबसे', 'सभी', 'साथ', 'साबुत', 'साभ', 'सारा', 'से', 'सो', 'संग', 'ही', 'हुआ', 'हुई', 'हुए', 'है',
    'हैं', 'हो', 'होता', 'होती', 'होते', 'होना', 'होने'
])  # Extend this list as needed
stopwords_english = set(["is", "and", "the", "of", "to", "in", "it", "that", "on", "for", "with", "as", "was"])  # Extend this list as needed

# Splitting input and label
x = data['title']  # Example: ["यह किताब अच्छी है", "This is a great book"]
y = data['label']  # Example: [1, 0]

# Preprocessing: Removing stopwords for mixed language dataset
corpus = []
for review in x:
    review = review.split()  # Tokenize into words
    # Remove stopwords for both languages
    review = [word for word in review if word not in stopwords_hindi and word.lower() not in stopwords_english]
    corpus.append(' '.join(review))  # Recombine into processed sentences

# Tokenization and Padding
voc_size = 5000  # Vocabulary size
tokenizer = Tokenizer(num_words=voc_size, oov_token="<OOV>")  # Tokenizer with out-of-vocabulary handling
tokenizer.fit_on_texts(corpus)  # Fit tokenizer on the processed corpus

# Convert text to numerical sequences
sequences = tokenizer.texts_to_sequences(corpus)

# Padding sequences to a fixed length
max_len = 20  # Maximum length for sequences
padded = pad_sequences(sequences, padding='post', maxlen=max_len)

# The padded array contains the tokenized and padded representation of the input data
print("Sample padded output:", padded[:5])  # Display the first 5 padded sequences


# Model Building
embed_dim = 40
model = Sequential([
    Embedding(voc_size, embed_dim, input_length=20),
    Bidirectional(LSTM(100)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train-Test Split
x = np.array(padded)
y = np.array(y)
trainX, testX, trainY, testY = train_test_split(x, y, test_size=0.3, random_state=0)

# Training
history = model.fit(trainX, trainY, epochs=10, validation_data=(testX, testY), batch_size=64)

# Predictions and Evaluation
pred = model.predict(testX)
binary_predictions = [1 if i >= 0.5 else 0 for i in pred]

print('Accuracy on testing set:', accuracy_score(binary_predictions, testY))
print('Precision on testing set:', precision_score(binary_predictions, testY))
print('Recall on testing set:', recall_score(binary_predictions, testY))

# Confusion Matrix and AUC
cm = confusion_matrix(testY, binary_predictions)
# Use model.predict instead of model.predict_proba
probs = model.predict(testX)[:, 0]  # Get the predicted probabilities

# Calculate AUC
auc = roc_auc_score(testY, probs)

print('Confusion Matrix:\n', cm)
print('AUC:', auc)



Sample padded output: [[ 601    1    1  266  872 1276  888 2520    0    0    0    0    0    0
     0    0    0    0    0    0]
 [ 802  460 3409  198    1    1  989  226    0    0    0    0    0    0
     0    0    0    0    0    0]
 [2137 2520    1  460 1037 3852    1  131    1 2706    1 1205    0    0
     0    0    0    0    0    0]
 [ 108  182  794    1 1862    1    1    3    1 1862 1900    2 2415    1
     0    0    0    0    0    0]
 [3853 2934 3661 3044    1    1    1    1   35   43 2934 3821    1    0
     0    0    0    0    0    0]]


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Epoch 1/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 56s 75ms/step - accuracy: 0.8182 - loss: 0.3624 - val_accuracy: 0.8993 - val_loss: 0.2261
Epoch 2/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 84s 78ms/step - accuracy: 0.9276 - loss: 0.1800 - val_accuracy: 0.9090 - val_loss: 0.2084
Epoch 3/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 50s 74ms/step - accuracy: 0.9439 - loss: 0.1422 - val_accuracy: 0.9131 - val_loss: 0.2125
Epoch 4/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 82s 75ms/step - accuracy: 0.9574 - loss: 0.1111 - val_accuracy: 0.9087 - val_loss: 0.2715
Epoch 5/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 82s 75ms/step - accuracy: 0.9676 - loss: 0.0864 - val_accuracy: 0.9093 - val_loss: 0.3215
Epoch 6/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 86s 82ms/step - accuracy: 0.9733 - loss: 0.0699 - val_accuracy: 0.9057 - val_loss: 0.3817
Epoch 7/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 58s 85ms/step - accuracy: 0.9804 - loss: 0.0537 - val_accuracy: 0.9020 - val_loss: 0.4380
Epoch 8/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 51s 76ms/step - accuracy: 0.9821 - loss: 0.0468 - 

In [8]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Bidirectional, Embedding, LSTM, Dropout, Dense
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, roc_auc_score

# Define custom Hindi stopwords
stopwords_hindi = [
    'अत', 'अपना', 'अपनी', 'अपने', 'अभी', 'अंदर', 'आदि', 'आप', 'इत्यादि', 'इन', 'इनका', 'इन्हीं', 'इन्हें', 'इन्हों',
    'इस', 'इसका', 'इसकी', 'इसके', 'इसमें', 'इसी', 'इसे', 'उन', 'उनका', 'उनकी', 'उनके', 'उनको', 'उन्हीं', 'उन्हें',
    'उन्हों', 'उस', 'उसके', 'उसी', 'उसे', 'एक', 'एवं', 'एस', 'ऐसे', 'और', 'कई', 'कर', 'करता', 'करते', 'करना',
    'करने', 'करें', 'कहते', 'कहा', 'का', 'काफ़ी', 'कि', 'कितना', 'किन्हें', 'किन्हों', 'किया', 'किर', 'किस',
    'किसी', 'किसे', 'की', 'कुछ', 'कुल', 'के', 'को', 'कोई', 'कौन', 'कौन', 'बही', 'बहुत', 'बाद', 'बाला', 'बिलकुल',
    'भी', 'भीतर', 'मगर', 'मानो', 'मे', 'में', 'यदि', 'यह', 'यहाँ', 'यही', 'या', 'यिह', 'ये', 'रखें', 'रहा', 'रहे',
    'ऱ्वासा', 'लिए', 'लिये', 'लेकिन', 'व', 'वग़ैरह', 'वर्ग', 'वह', 'वहाँ', 'वहीं', 'वाले', 'वुह', 'वे', 'सकता',
    'सकते', 'सबसे', 'सभी', 'साथ', 'साबुत', 'साभ', 'सारा', 'से', 'सो', 'संग', 'ही', 'हुआ', 'हुई', 'हुए', 'है',
    'हैं', 'हो', 'होता', 'होती', 'होते', 'होना', 'होने'
]

# Load dataset
data = pd.read_csv('/content/randomized_combined_dataset.csv')

# Drop null values
data = data.dropna()

# Splitting input and label
x = data['title']
y = data['label']

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.utils import pad_sequences

# Define stopwords for both Hindi and English
stopwords_hindi = set([
    'अत', 'अपना', 'अपनी', 'अपने', 'अभी', 'अंदर', 'आदि', 'आप', 'इत्यादि', 'इन', 'इनका', 'इन्हीं', 'इन्हें', 'इन्हों',
    'इस', 'इसका', 'इसकी', 'इसके', 'इसमें', 'इसी', 'इसे', 'उन', 'उनका', 'उनकी', 'उनके', 'उनको', 'उन्हीं', 'उन्हें',
    'उन्हों', 'उस', 'उसके', 'उसी', 'उसे', 'एक', 'एवं', 'एस', 'ऐसे', 'और', 'कई', 'कर', 'करता', 'करते', 'करना',
    'करने', 'करें', 'कहते', 'कहा', 'का', 'काफ़ी', 'कि', 'कितना', 'किन्हें', 'किन्हों', 'किया', 'किर', 'किस',
    'किसी', 'किसे', 'की', 'कुछ', 'कुल', 'के', 'को', 'कोई', 'कौन', 'कौन', 'बही', 'बहुत', 'बाद', 'बाला', 'बिलकुल',
    'भी', 'भीतर', 'मगर', 'मानो', 'मे', 'में', 'यदि', 'यह', 'यहाँ', 'यही', 'या', 'यिह', 'ये', 'रखें', 'रहा', 'रहे',
    'ऱ्वासा', 'लिए', 'लिये', 'लेकिन', 'व', 'वग़ैरह', 'वर्ग', 'वह', 'वहाँ', 'वहीं', 'वाले', 'वुह', 'वे', 'सकता',
    'सकते', 'सबसे', 'सभी', 'साथ', 'साबुत', 'साभ', 'सारा', 'से', 'सो', 'संग', 'ही', 'हुआ', 'हुई', 'हुए', 'है',
    'हैं', 'हो', 'होता', 'होती', 'होते', 'होना', 'होने'
])  # Extend this list as needed
stopwords_english = set(["is", "and", "the", "of", "to", "in", "it", "that", "on", "for", "with", "as", "was"])  # Extend this list as needed

# Splitting input and label
x = data['title']  # Example: ["यह किताब अच्छी है", "This is a great book"]
y = data['label']  # Example: [1, 0]

# Preprocessing: Removing stopwords for mixed language dataset
corpus = []
for review in x:
    review = review.split()  # Tokenize into words
    # Remove stopwords for both languages
    review = [word for word in review if word not in stopwords_hindi and word.lower() not in stopwords_english]
    corpus.append(' '.join(review))  # Recombine into processed sentences

# Tokenization and Padding
voc_size = 5000  # Vocabulary size
tokenizer = Tokenizer(num_words=voc_size, oov_token="<OOV>")  # Tokenizer with out-of-vocabulary handling
tokenizer.fit_on_texts(corpus)  # Fit tokenizer on the processed corpus

# Convert text to numerical sequences
sequences = tokenizer.texts_to_sequences(corpus)

# Padding sequences to a fixed length
max_len = 20  # Maximum length for sequences
padded = pad_sequences(sequences, padding='post', maxlen=max_len)

# The padded array contains the tokenized and padded representation of the input data
print("Sample padded output:", padded[:5])  # Display the first 5 padded sequences


# Model Building
embed_dim = 40
model = Sequential([
    Embedding(voc_size, embed_dim, input_length=20),
    Bidirectional(LSTM(100)),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Train-Test Split
x = np.array(padded)
y = np.array(y)
trainX, testX, trainY, testY = train_test_split(x, y, test_size=0.3, random_state=0)

# Training
history = model.fit(trainX, trainY, epochs=10, validation_data=(testX, testY), batch_size=64)

# Predictions and Evaluation
pred = model.predict(testX)
binary_predictions = [1 if i >= 0.5 else 0 for i in pred.flatten()]

print('Accuracy on testing set:', accuracy_score(testY, binary_predictions))
print('Precision on testing set:', precision_score(testY, binary_predictions))
print('Recall on testing set:', recall_score(testY, binary_predictions))

# Confusion Matrix and AUC
cm = confusion_matrix(testY, binary_predictions)
probs = pred.flatten()  # Get predicted probabilities
auc = roc_auc_score(testY, probs)

print('Confusion Matrix:\n', cm)
print('AUC:', auc)


Sample padded output: [[ 538    1    1  210  807 1205  823 2444    0    0    0    0    0    0
     0    0    0    0    0    0]
 [ 737  399 3332  143    1    1  922  170    0    0    0    0    0    0
     0    0    0    0    0    0]
 [2061 2444    1  399  969 3775    1   85    1 2630    1 1134    0    0
     0    0    0    0    0    0]
 [  66  131  729    1 1789    1    1    1 1789 1827 2339    1    0    0
     0    0    0    0    0    0]
 [3776 2858 3584 2967    1    1    1    1   19   22 2858 3744    1    0
     0    0    0    0    0    0]]
Epoch 1/10


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


679/679 ━━━━━━━━━━━━━━━━━━━━ 59s 80ms/step - accuracy: 0.8230 - loss: 0.3636 - val_accuracy: 0.9113 - val_loss: 0.2132
Epoch 2/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 60s 88ms/step - accuracy: 0.9290 - loss: 0.1753 - val_accuracy: 0.9139 - val_loss: 0.2105
Epoch 3/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 74s 77ms/step - accuracy: 0.9455 - loss: 0.1359 - val_accuracy: 0.9080 - val_loss: 0.2393
Epoch 4/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 84s 79ms/step - accuracy: 0.9566 - loss: 0.1123 - val_accuracy: 0.9094 - val_loss: 0.2540
Epoch 5/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 51s 75ms/step - accuracy: 0.9642 - loss: 0.0949 - val_accuracy: 0.9051 - val_loss: 0.3024
Epoch 6/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 81s 74ms/step - accuracy: 0.9727 - loss: 0.0706 - val_accuracy: 0.9054 - val_loss: 0.3494
Epoch 7/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 88s 82ms/step - accuracy: 0.9779 - loss: 0.0579 - val_accuracy: 0.9013 - val_loss: 0.3418
Epoch 8/10
679/679 ━━━━━━━━━━━━━━━━━━━━ 81s 81ms/step - accuracy: 0.9808 - loss: 0.0505 - val_accurac

In [9]:
from tensorflow.keras.callbacks import EarlyStopping

# Optimized Model
model = Sequential([
    Embedding(voc_size, embed_dim, input_length=15),  # Smaller embedding dimension and sequence length
    Bidirectional(LSTM(50)),  # Reduced LSTM units
    Dropout(0.2),
    Dense(32, activation='relu'),
    Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

# Early Stopping
early_stopping = EarlyStopping(monitor='val_loss', patience=2, restore_best_weights=True)

# Training
history = model.fit(trainX, trainY, epochs=5, validation_data=(testX, testY), batch_size=32, callbacks=[early_stopping])
# Predictions and Evaluation
pred = model.predict(testX)
binary_predictions = [1 if i >= 0.5 else 0 for i in pred.flatten()]

print('Accuracy on testing set:', accuracy_score(testY, binary_predictions))
print('Precision on testing set:', precision_score(testY, binary_predictions))
print('Recall on testing set:', recall_score(testY, binary_predictions))

# Confusion Matrix and AUC
cm = confusion_matrix(testY, binary_predictions)
probs = pred.flatten()  # Get predicted probabilities
auc = roc_auc_score(testY, probs)

print('Confusion Matrix:\n', cm)
print('AUC:', auc)


Epoch 1/5


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


1357/1357 ━━━━━━━━━━━━━━━━━━━━ 46s 30ms/step - accuracy: 0.8366 - loss: 0.3399 - val_accuracy: 0.9112 - val_loss: 0.2135
Epoch 2/5
1357/1357 ━━━━━━━━━━━━━━━━━━━━ 81s 30ms/step - accuracy: 0.9339 - loss: 0.1632 - val_accuracy: 0.9141 - val_loss: 0.2150
Epoch 3/5
1357/1357 ━━━━━━━━━━━━━━━━━━━━ 41s 30ms/step - accuracy: 0.9501 - loss: 0.1235 - val_accuracy: 0.9119 - val_loss: 0.2170
582/582 ━━━━━━━━━━━━━━━━━━━━ 5s 8ms/step
Accuracy on testing set: 0.9111624657387005
Precision on testing set: 0.9105343845816908
Recall on testing set: 0.9086438640585728
Confusion Matrix:
 [[8639  817]
 [ 836 8315]]
AUC: 0.972527949706753
